In [47]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [48]:
class Model1 (nn.Module):
    def __init__(self):
        super(Model1,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(5,1),
            nn.Tanh()
        )
    def forward(self,x):
        return self.model(x)

In [49]:
df = pd.read_json("data2/normal_vpm.json",orient="records",lines=True)


In [50]:
df.to_numpy()[:,:-1]

array([[2.31756216e-01, 3.00000000e+00, 1.00742900e-04, 1.50000000e+01,
        1.50000000e+01],
       [7.89317554e-01, 5.00000000e+00, 4.92996000e-05, 1.00000000e+01,
        0.00000000e+00],
       [7.89309901e-01, 5.00000000e+00, 4.93001000e-05, 1.00000000e+01,
        0.00000000e+00],
       ...,
       [6.36590830e-01, 2.00000000e+00, 2.44509000e-05, 1.50000000e+01,
        1.50000000e+01],
       [6.36581620e-01, 2.00000000e+00, 2.44513000e-05, 1.50000000e+01,
        1.50000000e+01],
       [6.36613789e-01, 3.00000000e+00, 3.66750000e-05, 1.50000000e+01,
        1.50000000e+01]])

In [51]:
train_x = torch.Tensor(df.to_numpy()[:10000,:-1])
train_Y = torch.Tensor(df.to_numpy()[:10000,-1].reshape(-1,1))
train_data = TensorDataset(train_x,train_Y)
train_dataLoader = DataLoader(train_data,batch_size=128,shuffle=True)

test_x = torch.Tensor(df.to_numpy()[10000:,:-1])
test_Y = torch.Tensor(df.to_numpy()[10000:,-1].reshape(-1,1))
test_dataset = TensorDataset(test_x,test_Y)
test_dataLoader = DataLoader(test_dataset,batch_size=128,shuffle=True)

In [52]:
model1 = Model1()
optimizer = optim.Adam(model1.parameters(),lr=0.07)
criterion = nn.MSELoss()

In [53]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model1.to(device)
print("Running model on:",device)

Running model on: cpu


In [54]:
epochs = 5

model1.train()
for epoch in tqdm(range(epochs)):
    train_loss = []
    for x, Y in train_dataLoader:
        x = x.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        out = model1(x)
        loss = criterion(out, Y)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    print(f"Loss: {np.average(train_loss):.3f}")

 40%|████      | 2/5 [00:00<00:00, 15.91it/s]Loss: 0.511
Loss: 0.325
Loss: 0.284
100%|██████████| 5/5 [00:00<00:00, 16.88it/s]Loss: 0.278



In [55]:
correct = 0
for x, Y in test_dataLoader:
    model1.eval()
    x = x.to(device)
    Y = Y.cpu()
    out=model1(x)
    out.cpu()
    for i,o in enumerate(out):
        if o * Y[i] > 0:
            correct += 1
print(correct,correct/len(test_dataset))


2486 0.903343023255814


In [56]:
model1.cpu()
print(model1(torch.Tensor(np.array([[0.5,4,0.00003,15,0],[0.5,4,0.00003,15,5],[0.5,4,0.00003,15,10],[0.5,4,0.00003,15,15],[0.5,4,0.00003,15,20]]))))
#print(model1(torch.Tensor(np.array([60000000,2,0,0]))))

tensor([[-1.0000],
        [-0.9998],
        [-0.9921],
        [-0.6718],
        [ 0.8137]], grad_fn=<TanhBackward>)


In [57]:
o = model1(test_x)
for i in o:
    print(i)

ckward>)
tensor([0.9912], grad_fn=<UnbindBackward>)
tensor([-0.9818], grad_fn=<UnbindBackward>)
tensor([-0.9818], grad_fn=<UnbindBackward>)
tensor([-0.9818], grad_fn=<UnbindBackward>)
tensor([-0.9818], grad_fn=<UnbindBackward>)
tensor([-0.9641], grad_fn=<UnbindBackward>)
tensor([-0.9839], grad_fn=<UnbindBackward>)
tensor([-0.9641], grad_fn=<UnbindBackward>)
tensor([-0.9641], grad_fn=<UnbindBackward>)
tensor([-0.9641], grad_fn=<UnbindBackward>)
tensor([-0.9641], grad_fn=<UnbindBackward>)
tensor([-0.9641], grad_fn=<UnbindBackward>)
tensor([-0.9641], grad_fn=<UnbindBackward>)
tensor([-0.9641], grad_fn=<UnbindBackward>)
tensor([-0.9641], grad_fn=<UnbindBackward>)
tensor([-0.9820], grad_fn=<UnbindBackward>)
tensor([-0.9820], grad_fn=<UnbindBackward>)
tensor([-0.9820], grad_fn=<UnbindBackward>)
tensor([-0.9820], grad_fn=<UnbindBackward>)
tensor([-0.9820], grad_fn=<UnbindBackward>)
tensor([-0.9820], grad_fn=<UnbindBackward>)
tensor([-0.9946], grad_fn=<UnbindBackward>)
tensor([-0.9820], grad_f

In [58]:
model1.model[0].weight

Parameter containing:
tensor([[-4.0598,  0.4056,  3.6904, -0.3461,  0.3904]], requires_grad=True)